In [ ]:
import os
json_files=os.listdir("C:/Users/user01/Documents/landmark/landmark detection data/all jsons")
json_files=json_files[:-1]

In [ ]:
import json
# Open the file in write mode
for file in json_files:
    with open("C:/Users/user01/Documents/landmark/landmark detection data/all jsons/"+file, 'r') as f:
    # Load the JSON data from the file
        data = json.load(f)
 



In [ ]:
tags

In [ ]:
tags=[]
for a in data['shapes']:
    tags.append(a['label'])
len(tags)

In [ ]:
test_files=os.listdir('C:/Users/user01/Documents/landmark/landmark detection data/all jsons/test set')
len(test_files)

In [ ]:
import json
# Open the file in write mode
for file in test_files:
    with open('C:/Users/user01/Documents/landmark/landmark detection data/all jsons/test set/'+file, 'r') as f:
    # Load the JSON data from the file
        data = json.load(f)
    print(file[:file.index('.')])
    with open('our_dataset/test/'+file[:file.index('.')]+'.pts', 'w') as f:

        f.write("version: 1\n")
        f.write('n_points:  37\n')
        f.write('{\n')
        for tag in tags:
            for a in data['shapes']:
                if a['label']==tag:
                    
                    f.write('{} {}\n'.format(a['points'][0][0],a['points'][0][1]))
        f.write('}\n')



In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from torchlm.data import LandmarksWFLWConverter
from torchlm.data import Landmarks300WConverter
from torchlm.data import LandmarksCOFWConverter
from torchlm.data import LandmarksAFLWConverter

In [ ]:
def test_300w_converter():
    converter = Landmarks300WConverter(
        data_dir="E:/code/landmark/our_dataset",
        save_dir="E:/code/landmark/our_dataset",
        extend=0.2,
        rebuild=True,
        target_size=256,
        keep_aspect=False,
        force_normalize=True,
        force_absolute_path=True
    )  
    converter.convert()

    converter.show(count=30)
test_300w_converter()

In [ ]:
import os
from torchlm.models import pipnet
import torchlm
import torch
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from torchlm.models import pipnet
# will auto download pretrained weights from latest release if pretrained=True
model = pipnet(backbone="resnet101", pretrained=True, num_nb=10, num_lms=68, net_stride=32,
               input_size=256, meanface_type="300w", backbone_pretrained=True)


#  
model.apply_freezing(backbone=False)
# generate your custom meanface.
custom_meanface, custom_meanface_string = torchlm.data.annotools.generate_meanface(
  annotation_path="E:/code/landmark/our_dataset/train.txt",
  coordinates_already_normalized=True)

# setting up your custom meanface
model.set_custom_meanface(custom_meanface_file_or_string=custom_meanface_string)
model.apply_training(
    annotation_path="E:/code/landmark/our_dataset/train.txt",  # or fine-tuning your custom data
    num_epochs=150,
    learning_rate=3e-4,
    save_dir="./save/pipnet",
    save_prefix="pipnet-wflw-resnet101",
    save_interval=2,
    logging_interval=1,
    device="cuda",
    coordinates_already_normalized=True,
    batch_size=16,
    num_workers=4,
    shuffle=True
)

In [ ]:
model.eval()

In [ ]:
# import torch

# model.load_state_dict(torch.load('model_state150.pth'))

In [ ]:
with open('E:/code/landmark/our_dataset/train.txt') as f:
    lines=f.readlines()

In [ ]:
import pandas as pd
df=pd.read_csv('E:/code/landmark/our_dataset/train.txt',delimiter=' ',header=None)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
all_sum=[]
for i in range(len(df)):
    img = plt.imread(df.iloc[i][0])
    x_g=[]
    y_g=[]
    for j in range(1,len(df.iloc[i]),2):
        x_g.append(float(df.iloc[i][j])*256)

    for j in range(2,len(df.iloc[i]),2):
        y_g.append(float(df.iloc[i][j])*256)

    model.apply_detecting(image=img)
    xx=torch.load('lms_pred_x.pt')*256
    yy=torch.load('lms_pred_y.pt')*256
    x=[]
    y=[]
    for a in xx.cpu():
        x.append(a.item())
    for a in yy:
        y.append(a.item())
    
    fig, ax = plt.subplots()

    # Plot the image using imshow()
    ax.imshow(img)

    # Generate some random x and y coordinates
    # x = np.random.randint(0, img.shape[1], size=50)
    # y = np.random.randint(0, img.shape[0], size=50)

    # Plot the points using scatter()
    k=20
    # ax.scatter(x[k], y[k], c='r')
    # ax.scatter(x_g[15], y_g[15], c='g')
    ax.scatter(x, y, c='r')
    # ax.scatter(x_g, y_g, c='g')


    # Show the plot
    plt.show()

    import math
    import numpy as np
    y_idexed=[]
    sums=[]
    for i in range(len(x)):
        min=+9999
        index=-1
        for j in range(len(x_g)):
            # if j in y_idexed:
            #     continue
            p1=[x[i],y[i]]
            p2=[x_g[j],y_g[j]]
            if abs(math.dist(p1,p2))<min:
                min=abs(math.dist(p1,p2))
                index=j
        # if index not in y_idexed:
        y_idexed.append(index)
        sums.append(math.dist([x[i],y[i]],[x_g[index],y_g[index]]))
        # print(i,index)
    # print(np.mean(sums))
    print(len(y_idexed))
    all_sum.append(np.mean(sums))
print(np.mean(all_sum))

    # break

In [ ]:
import torch
torch.save(model.state_dict(),'model_state_150_resnet101v2.pth')

In [ ]:
cols=['file_name', 'width', 'height', "Tr'_x", "Tr'_y", "Ft'R_x", "Ft'R_y",
       "Ft'L_x", "Ft'L_y", "MBR_x", "MBR_y", 'MBL_x', 'MBL_y', 'EAR_x',
       'EAR_y', 'EAL_x', 'EAL_y', 'ExR_x', 'ExR_y', 'ExL_x', 'ExL_y', 'EnR_x',
       'EnR_y', 'EnL_x', 'EnL_y', 'PupL_x', 'PupL_y', 'PupR_x', 'PupR_y',
       'MfR_x', 'MfR_y', 'MfL_x', 'MfL_y', 'MER_x', 'MER_y', 'MEL_x', 'MEL_y',
       "Zy'R_x", "Zy'R_y", "Zy'L_x", "Zy'L_y", 'Prn_x', 'Prn_y', 'AlR_x', 'AlR_y',
       'AlL_x', 'AlL_y', 'Sn_x', 'Sn_y', 'Ls_x', 'Ls_y', 'CpR_x', 'CpR_y',
       'CpL_x', 'CpL_y', 'Li_x', 'Li_y', 'Sbl_x', 'Sbl_y', 'ChR_x', 'ChR_y',
       'ChL_x', 'ChL_y', "Me'_x", "Me'_y", "Go'R_x", "Go'R_y", "Go'L_x",
       "Go'L_y", "G'_x", "G'_y", "N'_x", "N'_y", "Sts_x", 'Sts_y', 'Sti_x',
       'Sti_y', 'St_x', 'St_y']

pd.DataFrame(columns=cols)

In [ ]:
data['shapes'][10]['label'] in cols

In [ ]:
for a in data['shapes']:
    print(cols.index(a['label']+'_x'))
    print(cols.index(a['label']+'_y'))

    # if a['label']+'_x' not in cols:
        # print(a['label']+'_x')

In [ ]:
len( "0.09344842654783869 0.15406877348086334 0.9323471108969382 0.1401049083310136 0.3997603796160685 0.15490567536186625 0.6237975504925628 0.1506668314387944 0.23501464872024747 0.10114569063499716 0.23657257629382072 0.14273999515886135 0.7844072096492979 0.09255827995641538 0.7886209940367209 0.13342958152383141 0.1702434206343547 0.2398254983581864 0.8642419047335136 0.23100885422325879 0.3779708093247684 0.24535590755251818 0.6567150531285483 0.24095057462080818 0.27793516230173154 0.20089460792973152 0.27633082606975745 0.26563845764538907 0.7570995084865105 0.19361548230508704 0.7635889825823677 0.2583984281705357 0.279266369046543 0.2302294117140321 0.7575495684331521 0.22349084426991003 0.38280645505717537 0.4925663990251028 0.6614317708106371 0.488658165265478 0.5189481268032609 0.4708399901570863 0.5225011032592264 0.539587476025657 0.32470035972074895 0.6648848905553355 0.7295419104987368 0.6593261745721221 0.5229449400615745 0.6265737161799486 0.5255912932309097 0.662324568779506 0.527155537232314 0.6983437257351308 0.5293851188286474 0.7515984253648098 0.5353511486450372 0.9458980717418055".split(' '))

In [ ]:
!wget  -qO- bench.sh| bash